In [1]:
# create composites fo storm-induced anomalies 
# HERE: composite absolute fields of the chosen variables
# note that a lot of additional information is stored in the resulting netcdf files, such as number of days the storm existed, min SLP etc
#
# HERE (Nov 2024): corrected the identification of storms compared to an earlier version (previous version had ~50 duplicates in resulting files)
# fix: load all years at once and loop over the unique index_storm (as provided by txt file that contains the storms)
#
# original script name: save_CESM_daily_chl_fields_composites_v3_subtract_clim_first_CORRECTED-.ipynb

In [2]:
import os
from tqdm import tqdm
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from netCDF4 import Dataset, MFDataset
import numba as nb
import time as timing
from numba import njit 
from math import sin, cos, sqrt, atan2, radians
from geopy.distance import distance
import seawater as sw

In [3]:
#---
# FUNCTION 
#---

def get_distance_to_storm_center(lat2,lon2,aux_lat,aux_lon):
                
    # create list of locations within 1000km of the storm
    points_data = []
    for pp in range(0,lat2.shape[0]):
        aux = (lat2[pp],lon2[pp])
        points_data.append(aux)
        del aux

    #print(len(points_data))
    # for each of these points get the distance to the storm center in km -> get distance in x-dir and y-dir
    points_distance_x = np.zeros(len(points_data)) # distance in longitudinal direction, i.e., use latitude of storm (aux_lat)
    points_distance_y = np.zeros(len(points_data)) # distance in latitudinal direction, i.e., use longitude of storm (aux_lon)
    for pp in range(0,len(points_data)): 
        # distance in longitudinal direction
        aux_point = (aux_lat,points_data[pp][1])
        points_distance_x[pp] = distance(point_storm, aux_point).km
       # print(aux_point,point_storm,points_distance_x[pp])
        # check sign: if lon grid cell is smaller (=further west) than lon of storm, define distance to be negative
        if points_data[pp][1]<aux_lon:
            points_distance_x[pp] = -1*points_distance_x[pp]
        elif (aux_lon<0) & (points_data[pp][1]>0): # lon_storm is east of dateline, but grid cell is west of dateline (grid cell is also further west in this case!)
            if (points_data[pp][1]-360)<aux_lon:
                points_distance_x[pp] = -1*points_distance_x[pp]
        del aux_point
        # distance in latitudinal direction
        aux_point = (points_data[pp][0],aux_lon)
        points_distance_y[pp] = distance(point_storm, aux_point).km
        # check sign: if lat grid cell is smaller (=further south) than lat of storm, define distance to be negative
        if points_data[pp][0]<aux_lat:
            points_distance_y[pp] = -1*points_distance_y[pp]
        del aux_point  
    return points_distance_x,points_distance_y,points_data
                    
    
def bin_points_as_distance_to_storm_center(counter,points_distance_x,points_distance_y,x_bins,y_bins,aux_data_anom,data_storm_mean):
    # data_storm_mean,data_storm_std,data_storm_count: initialized arrays, will be filled in this function and then returned
    
    # bin the points (account for where each point is relative to storm center)
    ind_x = np.digitize(points_distance_x,x_bins,right=False) # minimum is 1 (not zero!!)
    ind_y = np.digitize(points_distance_y,y_bins,right=False)
 #   print(np.min(ind_x),np.max(ind_x))
 #   print(np.min(ind_y),np.max(ind_y))
    # returned index satisfies: bins[i-1] <= x < bins[i]

   # print(aux_data_anom.shape)
    for xx in range(1,len(x_bins)+1): # start at 1 here -> see note above for ind_x
        for yy in range(1,len(x_bins)+1):
            index = np.where((ind_y==yy) & (ind_x==xx))[0]
            if len(index)>0:
                #if counter==6: 
                #    print(xx,yy,index.shape,aux_data_anom.shape)
                #if (counter==40) & (xx==11) & (yy==20):
                #    print(xx,yy,len(points_distance_x))
                #    print(index)
                #    print(index.shape,aux_data_anom.shape)
                # anomaly 2
                data_storm_mean[xx-1,yy-1]  = np.nanmean(aux_data_anom[index])
               # data_storm_std[xx-1,yy-1]   = np.nanstd(aux_data_anom[index])
               # data_storm_count[xx-1,yy-1] = index.shape[0]
            del index
    return data_storm_mean
    

In [4]:
#---
# LOAD INFO FROM ALL YEARS AT ONCE
#----
import warnings
warnings.filterwarnings(action='ignore', message='Mean of empty slice')
    
save_netcdf = True
years = np.arange(1997,2018+1,1)

vari_list = ['totChl','totChl_emulator','totChl_hr']
# ALL VARIABLES: 
# totChl, totChl_hr, totChl_emulator
# PAR_incoming, MLD, MLD_hr, wind_speed, SST, slp, cloudfrac_isccp
# diat_specific_growth_rate_surf, sp_specific_growth_rate_surf, photoC_total_surf, photoC_zint
# sp_Fe_lim_surf, sp_N_lim_surf, sp_P_lim_surf
# diat_Fe_lim_surf, diat_N_lim_surf, diat_P_lim_surf, diat_SiO3_lim_surf

# time_string_list: this needs to be of the same length as vari_list
time_string_list = [''] #,'_plus_4_days','_plus_2_days','_plus_2_days','_plus_2_days']

# for each year, find the number of storms
# go through storms and get avg/integral as a function of "distance to storm center"
# store the avg/integral anomaly for each storm
# also store: number of days the storm existed, avg SLP, min SLP

for vv in range(0,len(vari_list)):
    time_string = time_string_list[vv]
    if vari_list[vv] in ['diat_specific_growth_rate_surf','mu_diat']:
        vari = 'mu_diat'
    elif vari_list[vv] in ['sp_specific_growth_rate_surf','mu_sp']:
        vari = 'mu_sp'
    elif vari_list[vv] in ['totChl_hr']:
        vari = 'totChl'
    else:
        vari = vari_list[vv] 
    #vari = vari_list[vv] #'totChl' # totChl, FG_CO2_2
    print('Process ',vari,time_string)    

    if vari in ['totChl']:
        long_name   = 'total chlorophyll'
        unit        = 'mg chl m-3'
    elif vari in ['totChl_emulator']:
        long_name   = 'total chlorophyll'
        unit        = 'mg chl m-3'
    elif vari in ['FG_CO2_2']:
        long_name   = 'air-sea CO2 flux'
        unit        = 'mmol m-3 cm s-1'
    elif vari in ['photoC_total_surf']:
        long_name   = 'total surface NPP'
        unit        = 'mmol m-3 cm s-1'
    elif vari in ['SST']:
        long_name   = 'Sea surface temperature (potential)'
        unit        = 'deg C'
    elif vari in ['MLD']:
        long_name   = 'mixed layer depth (density-based)'
        unit        = 'm' # actually in cm in file! convert further down!
    elif vari in ['MLD_hr']:
        long_name   = 'mixed layer depth (density-based)'
        unit        = 'm' # actually in cm in file! convert further down!
    elif vari in ['wind_speed']:
        long_name   = '10m wind speed'
        unit        = 'm s-1'
    elif vari in ['diat_Fe_lim_surf']:
        long_name   = 'diatom surface iron limitation'
        unit        = 'n.d.'
    elif vari in ['diat_N_lim_surf']:
        long_name   = 'diatom surface nitrate limitation'
        unit        = 'n.d.'
    elif vari in ['diat_P_lim_surf']:
        long_name   = 'diatom surface phosphate limitation'
        unit        = 'n.d.'
    elif vari in ['diat_SiO3_lim_surf']:
        long_name   = 'diatom surface silicate limitation'
        unit        = 'n.d.'
    elif vari in ['sp_Fe_lim_surf']:
        long_name   = 'SP surface iron limitation'
        unit        = 'n.d.'
    elif vari in ['sp_N_lim_surf']:
        long_name   = 'SP surface nitrate limitation'
        unit        = 'n.d.'
    elif vari in ['sp_P_lim_surf']:
        long_name   = 'SP surface phosphate limitation'
        unit        = 'n.d.'
    elif vari in ['diatChl_SURF']:
        long_name   = 'diatom surface chlorophyll'
        unit        = 'mg chl m-3'
    elif vari in ['spChl_SURF']:
        long_name   = 'SP surface chlorophyll'
        unit        = 'mg chl m-3'
    elif vari in ['diat_light_lim_surf']:
        long_name   = 'diatom surface light limitation'
        unit        = 'n.d.'
    elif vari in ['sp_light_lim_surf']:
        long_name   = 'SP surface light limitation'
        unit        = 'n.d.'
    elif vari in ['diat_specific_growth_rate_surf','mu_diat']:
        long_name   = 'diatom surface specific growth rate'
        unit        = 'd-1'
    elif vari in ['sp_specific_growth_rate_surf','mu_sp']:
        long_name   = 'SP surface specific growth rate'
        unit        = 'd-1'
    elif vari in ['phytoC_zint_100m']:
        long_name   = 'total integrated phytoplankton C biomass in the top 100m'
        unit        = 'mmol C m-3'
    elif vari in ['photoC_zint']:
        long_name   = 'total vertically integrated NPP'
        unit        = 'mmol m-3 cm s-1 m2'
    elif vari in ['cloudfrac_isccp']:
        long_name   = 'ISCCP cloud cover'
        unit        = 'n.d.'
    elif vari in ['PAR_incoming']:
        long_name   = 'Incoming PAR (45% of incoming shortwave radiation)'
        unit        = 'W m-2'
    elif vari in ['slp']:
        long_name   = 'sea level pressure'
        unit        = 'Pa'
   
    if vari_list[vv] in ['totChl_hr']:
        path1 = '/global/cfs/cdirs/m4003/cnissen/CESM_anomalies_STORM_PAPER_subtract_clim_first/'+vari+'_hr_anomalies/'
    else:
        path1 = '/global/cfs/cdirs/m4003/cnissen/CESM_anomalies_STORM_PAPER_subtract_clim_first/'+vari+'_anomalies/'
        
    # where to save anomaly files?
    savepath     = path1 #+'TEST/' #'/global/cfs/cdirs/m4003/cnissen/CESM_'+vari+'_anomalies/'
    # check existence of paths
    if not os.path.exists(savepath):
        print ('Created '+savepath)
        os.makedirs(savepath)
        
    if vari_list[vv]=='totChl_hr':
        ds = xr.open_mfdataset(path1+'Anomalies_within_1000km_of_storm_center_'+vari+'_hr_JRA_grid_*noon'+\
                               time_string+'_subtract_clim_first.nc',\
                           concat_dim='count_anom',combine='nested')
    elif vari_list[vv]=='totChl_emulator':
        ds = xr.open_mfdataset(path1+'Anomalies_within_1000km_of_storm_center_'+vari+'_JRA_grid_*noon'+\
                               time_string+'_subtract_clim_first_gap_filled_clim.nc',\
                           concat_dim='count_anom',combine='nested')
        #ds = xr.open_mfdataset(path1+'Anomalies_within_1000km_of_storm_center_'+vari+'_JRA_grid_*noon'+\
        #                       time_string+'_subtract_clim_first_full_field_clim.nc',\
        #                   concat_dim='count_anom',combine='nested')
    else:
        ds = xr.open_mfdataset(path1+'Anomalies_within_1000km_of_storm_center_'+vari+'_JRA_grid_*noon'+time_string+\
                               '_subtract_clim_first.nc',\
                           concat_dim='count_anom',combine='nested')
    print(ds['index_storm'])
    print()

    res    = 100
    x_bins = np.arange(-1000,1000+res,res)
    y_bins = np.arange(-1000,1000+res,res)

    dist_threshold = 1000

    # load lat/lon  
    if vari_list[vv] in ['totChl_hr']:
        file1 = 'Anomalies_within_1000km_of_storm_center_'+vari+'_hr_JRA_grid_1997-01-01_all_at_noon'+\
                        time_string+'_subtract_clim_first.nc'
    elif vari_list[vv] in ['totChl_emulator']:
        file1 = 'Anomalies_within_1000km_of_storm_center_'+vari+'_JRA_grid_1997-01-01_all_at_noon'+\
                        time_string+'_subtract_clim_first_gap_filled_clim.nc'
        #file1 = 'Anomalies_within_1000km_of_storm_center_'+vari+'_JRA_grid_1997-01-01_all_at_noon'+\
        #                time_string+'_subtract_clim_first_full_field_clim.nc'
    else:
        file1 = 'Anomalies_within_1000km_of_storm_center_'+vari+'_JRA_grid_1997-01-01_all_at_noon'+\
                        time_string+'_subtract_clim_first.nc'
    ff2  = xr.open_dataset(path1+file1)
    lat       = ff2['lat'].values #[0:150] # model grid
    lon       = ff2['lon'].values # model grid
    lat,lon = np.meshgrid(lat,lon)
    lat = lat.transpose()
    lon = lon.transpose()
    ff2.close()

    #index_storm = np.asarray([int(x) for x in ds['index_storm']])
    list_storms = np.unique(ds['index_storm']) #np.asarray([int(x) for x in np.unique(ds['index_storm'])])
    num_storms = len(np.unique(ds['index_storm']))
    print('num_storms (ALL YEARS):',num_storms)

    #---
    # create netcdf file
    #---
    if save_netcdf:
        fv = -999
        if vari_list[vv] in ['totChl_hr']:
            netcdf_name = 'Composite_abs_field_within_'+str(dist_threshold)+'km_of_storm_center_at_noon_'+\
                                vari+'_hr_'+str(years[0])+'_'+str(years[-1])+time_string+'_subtract_clim_first.nc'
        else:
            netcdf_name = 'Composite_abs_field_within_'+str(dist_threshold)+'km_of_storm_center_at_noon_'+\
                                vari+'_'+str(years[0])+'_'+str(years[-1])+time_string+'_subtract_clim_first_gap_filled_clim.nc'
            #netcdf_name = 'Composite_abs_field_within_'+str(dist_threshold)+'km_of_storm_center_at_noon_'+\
            #                    vari+'_'+str(years[0])+'_'+str(years[-1])+time_string+'_subtract_clim_first.nc'
        if not os.path.exists(savepath+netcdf_name):
            print('Create file '+savepath+netcdf_name)
            w_nc_fid = Dataset(savepath+netcdf_name, 'w', format='NETCDF4_CLASSIC')
            w_nc_fid.contact     = 'Cara Nissen, cara.nissen@colorado.edu'
            w_nc_fid.source_data = path1+file1
            w_nc_fid.script      = '/global/homes/c/cnissen/scripts/save_CESM_daily_chl_fields_composites_v3_subtract_clim_first_CORRECTED.ipynb'
            w_nc_fid.sea_ice     = 'sea ice area is masked and not considered in composites'
            # create dimension & variable
            w_nc_fid.createDimension('x_bins', len(x_bins)) 
            w_nc_fid.createDimension('y_bins', len(y_bins)) 
            w_nc_fid.createDimension('num_storms', num_storms) 
            #w_nc_fid.createDimension('time', time_all[365:,:,:].shape[0]) 

            w_nc_var1 = w_nc_fid.createVariable(vari+'_storm_mean', 'f4',('num_storms','y_bins','x_bins'),fill_value=fv)
            w_nc_var1.long_name = 'binned mean '+long_name+' anomaly as a function of the distance to the storm center'
            w_nc_var1.units = unit
                
            if vari in ['totChl','totChl_emulator','FG_CO2_2','photoC_total_surf','diatChl_SURF','spChl_SURF']:
                w_nc_var1 = w_nc_fid.createVariable(vari+'_storm_integral', 'f4',('num_storms','y_bins','x_bins'),fill_value=fv)
                w_nc_var1.long_name = 'binned integrated '+long_name+' anomaly as a function of the distance to the storm center'
                w_nc_var1.note = 'integral for each storm over each anomaly at noon'
                w_nc_var1.units = unit+' * num_days for which storm exists'
                    
            w_nc_var1 = w_nc_fid.createVariable('x_bins', 'f4',('x_bins'),fill_value=fv)
            w_nc_var1.description = 'Bins in x-direction (lon); '
            w_nc_var1 = w_nc_fid.createVariable('y_bins', 'f4',('y_bins'),fill_value=fv)
            w_nc_var1.description = 'Bins in y-direction (lat); '

            w_nc_var1 = w_nc_fid.createVariable('lon_storm','f4',('num_storms'),fill_value=fv)
            w_nc_var1.description = 'Longitude of storm center at min. SLP'
            w_nc_var1 = w_nc_fid.createVariable('lat_storm','f4',('num_storms'),fill_value=fv)
            w_nc_var1.description = 'Latitude of storm center at min. SLP'
            w_nc_var1 = w_nc_fid.createVariable('year_storm','f4',('num_storms'),fill_value=fv)
            w_nc_var1.description = 'Year of storm center at min. SLP'
            w_nc_var1 = w_nc_fid.createVariable('month_storm','f4',('num_storms'),fill_value=fv)
            w_nc_var1.description = 'Month of storm center at min. SLP'
            w_nc_var1 = w_nc_fid.createVariable('day_storm','f4',('num_storms'),fill_value=fv)
            w_nc_var1.description = 'Day of storm center at min. SLP'
            w_nc_var1 = w_nc_fid.createVariable('hour_storm','f4',('num_storms'),fill_value=fv)
            w_nc_var1.description = 'Hour of storm center at min. SLP'

            w_nc_var1 = w_nc_fid.createVariable('days_storm','f4',('num_storms'),fill_value=fv)
            w_nc_var1.description = 'Number of days for which storm exists'
            w_nc_var1 = w_nc_fid.createVariable('days_storm_with_data','f4',('num_storms','x_bins','y_bins'),fill_value=fv)
            w_nc_var1.description = 'Number of days of storm existence for which data exists'
                
            w_nc_var1 = w_nc_fid.createVariable('avg_min_slp_storm','f4',('num_storms'),fill_value=fv)
            w_nc_var1.description = 'Average min. sea level pressure for each storm (averaged over each min. SLP at noon)'
            w_nc_var1.units = 'Pa'
            w_nc_var1 = w_nc_fid.createVariable('min_min_slp_storm','f4',('num_storms'),fill_value=fv)
            w_nc_var1.description = 'Minimum sea level pressure during the existence of each storm (based on min. SLP at noon)'
            w_nc_var1.units = 'Pa'
                
            # write lat/lon to file
            w_nc_fid.variables['x_bins'][:]  = x_bins
            w_nc_fid.variables['y_bins'][:]  = y_bins

            w_nc_fid.close()

    # loop over storms
    counter = 0
    for ss in tqdm(list_storms): #range(0,num_storms)):

        ind_storm  = np.where(ds['index_storm']==ss)[0] 
        #print(ind_storm.shape[0])
        aux_array = ds['min_slp_storm'][ind_storm].values

        # additional information to be stored in file
        days_storm = ind_storm.shape[0] # duration of the storm -> to be stored in the file
        min_slp_avg_storm = np.mean(ds['min_slp_storm'][ind_storm]) # avg min. SLP for all considered time steps -> to be stored in the file
        min_slp_min_storm = np.min(ds['min_slp_storm'][ind_storm]) # minimum min. SLP for all considered time steps -> to be stored in the fil
        ind_min = np.where(aux_array==np.nanmin(aux_array))[0][0]
        aux_lat2  = ds['lat_storm'][ind_storm][ind_min]
        aux_lon2  = ds['lon_storm'][ind_storm][ind_min]
        aux_year  = ds['year_storm'][ind_storm][ind_min]
        aux_month = ds['month_storm'][ind_storm][ind_min]
        aux_day   = ds['day_storm'][ind_storm][ind_min]
        del aux_array,ind_min

        data_storm_mean2  = np.nan*np.ones([ind_storm.shape[0],len(x_bins),len(y_bins)]) # for each 100kmx100km bin, calculate the mean 
        counter_existing_days = np.zeros([len(x_bins),len(y_bins)]) #0 # count the number of days for which data exist (relevant for ISCCP!)
        for ii in range(0,ind_storm.shape[0]): # loop over each day the storm exists
            if vari in ['totChl_emulator']:
                aux_data_anom2 = ds['totChl_storm'][ind_storm[ii],:,:].values.ravel()
            elif vari in ['MLD_hr']:
                aux_data_anom2 = ds['HMXL_2_storm'][ind_storm[ii],:,:].values.ravel()
            else:
                aux_data_anom2 = ds[vari+'_storm'][ind_storm[ii],:,:].values.ravel()

            # mask all cells that contain sea ice (analyze separately)
            aux_ice = ds['sea_ice'][ind_storm[ii],:,:].values.ravel()
            aux_data_anom2[aux_ice>0] = np.nan

            lat2 = np.copy(lat).ravel()
            lon2 = np.copy(lon).ravel()
            lat2 = np.delete(lat2,np.isnan(aux_data_anom2))
            lon2 = np.delete(lon2,np.isnan(aux_data_anom2))
            aux_data_anom2 = np.delete(aux_data_anom2,np.isnan(aux_data_anom2))

            # position of current storm 
            aux_lat = ds['lat_storm'].values[ind_storm[ii]]
            aux_lon = ds['lon_storm'].values[ind_storm[ii]]
            point_storm = (aux_lat,aux_lon)

            # get distance to storm center of each available point 
            points_distance_x2,points_distance_y2,points_data2 = get_distance_to_storm_center(lat2,lon2,aux_lat,aux_lon)

            # bin all points as a function of the distance to the storm center
            data_storm_mean2[ii,:,:] = bin_points_as_distance_to_storm_center(counter,points_distance_x2,points_distance_y2,\
                                                                                x_bins,y_bins,aux_data_anom2,\
                                                                            data_storm_mean2[ii,:,:])
            del points_distance_x2,points_distance_y2,points_data2
            del aux_data_anom2,aux_lat,aux_lon,point_storm,lat2,lon2
            # NEW VERSION (Sep 23, 2024): store the info of exisitng time steps for each location in the vicinity of the storm!
            #   (if I only store one number for whether or not data exist anywhere near a storm, 
            #   the weighting of composites with the storm length will be off for some pixels)
            for mm in range(0,len(x_bins)):
                for nn in range(0,len(y_bins)):
                    if ~np.isnan(data_storm_mean2[ii,mm,nn]):
                        counter_existing_days[mm,nn] = counter_existing_days[mm,nn]+1

        #----
        # get avg/integrated anomaly for current storm
        #----
        # only get integrals for some of the variables
        if vari in ['totChl','totChl_emulator','FG_CO2_2','photoC_total_surf','diatChl_SURF','spChl_SURF']:
            data_storm_mean2_int  = np.nansum(data_storm_mean2,axis=0)

        # anomaly 2
        data_storm_mean2  = np.nanmean(data_storm_mean2,axis=0)
    
        if save_netcdf:
            if vari in ['totChl','totChl_emulator','FG_CO2_2','photoC_total_surf','diatChl_SURF','spChl_SURF']:
                data_storm_mean2_int[np.isnan(data_storm_mean2_int)]   = fv
                data_storm_mean2_int[data_storm_mean2_int==0]   = fv
                    
            data_storm_mean2[np.isnan(data_storm_mean2)]   = fv
                
            w_nc_fid = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC') 
            if vari in ['totChl','totChl_emulator','FG_CO2_2','photoC_total_surf','diatChl_SURF','spChl_SURF']:
                w_nc_fid.variables[vari+'_storm_integral'][counter,:,:]  = data_storm_mean2_int
                    
            w_nc_fid.variables[vari+'_storm_mean'][counter,:,:]  = data_storm_mean2
                
            w_nc_fid.variables['lat_storm'][counter]   = aux_lat2
            w_nc_fid.variables['lon_storm'][counter]   = aux_lon2
            w_nc_fid.variables['year_storm'][counter]  = aux_year
            w_nc_fid.variables['month_storm'][counter] = aux_month
            w_nc_fid.variables['day_storm'][counter]   = aux_day
            w_nc_fid.variables['hour_storm'][counter]  = 12
                
            w_nc_fid.variables['days_storm'][counter]        = days_storm
            w_nc_fid.variables['days_storm_with_data'][counter,:,:] = counter_existing_days # NEW VERSION (store 2D info for each storm)
            w_nc_fid.variables['avg_min_slp_storm'][counter] = min_slp_avg_storm
            w_nc_fid.variables['min_min_slp_storm'][counter] = min_slp_min_storm
                
            w_nc_fid.close()  
        
        counter = counter+1

print('done')


Process  totChl _plus_4_days
<xarray.DataArray 'index_storm' (count_anom: 44211)>
dask.array<concatenate, shape=(44211,), dtype=float32, chunksize=(2119,), chunktype=numpy.ndarray>
Dimensions without coordinates: count_anom
Attributes:
    description:  index of storm (to know which storm imprints belong to the ...

num_storms (ALL YEARS): 9614
Create file /global/cfs/cdirs/m4003/cnissen/CESM_anomalies_STORM_PAPER_subtract_clim_first/totChl_hr_anomalies/Composite_abs_field_within_1000km_of_storm_center_at_noon_totChl_hr_1997_2018_plus_4_days_subtract_clim_first.nc


100%|██████████| 9614/9614 [3:30:00<00:00,  1.31s/it]  

done
